In [4]:
#    This file is part of DEAP.
#
#    DEAP is free software: you can redistribute it and/or modify
#    it under the terms of the GNU Lesser General Public License as
#    published by the Free Software Foundation, either version 3 of
#    the License, or (at your option) any later version.
#
#    DEAP is distributed in the hope that it will be useful,
#    but WITHOUT ANY WARRANTY; without even the implied warranty of
#    MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the
#    GNU Lesser General Public License for more details.
#
#    You should have received a copy of the GNU Lesser General Public
#    License along with DEAP. If not, see <http://www.gnu.org/licenses/>.

!pip install deap
import random

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

IND_INIT_SIZE = 5   # initial knapsack item number
MAX_ITEM = 50       # knapsack capacity for item number
MAX_WEIGHT = 50     # knapsack capacity for item weight
NBR_ITEMS = 20      # item index - sampling w/o replacement

# To assure reproducibility, the RNG seed is set prior to the items
# dict initialization. It is also seeded in main().
random.seed(64)

# Create the item dictionary: item name is an integer, and value is
# a (weight, value) 2-tuple.
items = {}
# Create random items and store them in the items' dictionary.
for i in range(NBR_ITEMS):
    items[i] = (random.randint(1, 10), random.uniform(0, 100))

creator.create("Fitness", base.Fitness, weights=(-1.0, 1.0))
creator.create("Individual", set, fitness=creator.Fitness)

# [?] - what if we use list instead of set to represent a knapsack?

toolbox = base.Toolbox()

# Attribute generator
toolbox.register("attr_item", random.randrange, NBR_ITEMS)

# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual,
    toolbox.attr_item, IND_INIT_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evalKnapsack(individual):
    weight = 0.0
    value = 0.0
    for item in individual:
        weight += items[item][0]
        value += items[item][1]
    if len(individual) > MAX_ITEM or weight > MAX_WEIGHT:
        return 10000, 0             # Ensure overweighted bags are dominated
    return weight, value

def cxSet(ind1, ind2):
    """Apply a crossover operation on input sets. The first child is the
    intersection of the two sets, the second child is the difference of the
    two sets.
    """
    temp = set(ind1)                # Used in order to keep type
    ind1 &= ind2                    # Intersection (inplace)
    ind2 ^= temp                    # Symmetric Difference (inplace)
    return ind1, ind2

def mutSet(individual):
    """Mutation that pops or add an element."""
    if random.random() < 0.5:
        if len(individual) > 0:     # We cannot pop from an empty set
            individual.remove(random.choice(sorted(tuple(individual))))
    else:
        individual.add(random.randrange(NBR_ITEMS))
    return individual,

toolbox.register("evaluate", evalKnapsack)
toolbox.register("mate", cxSet)
toolbox.register("mutate", mutSet)
toolbox.register("select", tools.selNSGA2)

def main():
    random.seed(64)
    NGEN = 50
    MU = 50
    LAMBDA = 100
    CXPB = 0.7
    MUTPB = 0.2

    pop = toolbox.population(n=MU)
    hof = tools.ParetoFront()
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean, axis=0)
    stats.register("std", numpy.std, axis=0)
    stats.register("min", numpy.min, axis=0)
    stats.register("max", numpy.max, axis=0)

    # (mu, lambda)-ES: evolution strategies， where children replace parents.
    # (mu + lambda)-ES: evolution strategies, where children and parents are added to the population.
    algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN, stats,
                              halloffame=hof)

    return pop, stats, hof

if __name__ == "__main__":
    main()

gen	nevals	avg                        	std                      	min                      	max                        
0  	50    	[ 22.78       210.00877715]	[ 5.88316241 71.09309649]	[10.         49.69958685]	[ 38.         345.35491309]
1  	87    	[  9.96       145.20790666]	[ 10.61312395 139.1868852 ]	[0. 0.]                  	[ 45.         414.48478381]
2  	91    	[ 3.26       61.20087478]  	[  7.44797959 125.53892809]	[0. 0.]                  	[ 28.         414.48478381]
3  	88    	[ 4.6        84.51641114]  	[  8.57438044 140.51459866]	[0. 0.]                  	[ 28.         414.48478381]
4  	92    	[ 2.4        52.24310488]  	[  5.82065288 108.88598622]	[0. 0.]                  	[ 28.         414.48478381]
5  	87    	[ 3.66       74.97342258]  	[  6.99316809 127.02866009]	[0. 0.]                  	[ 28.         414.48478381]
6  	82    	[  5.3        111.43072783]	[  7.61117599 142.76899122]	[0. 0.]                  	[ 28.         414.48478381]
7  	90    	[ 3.38       76.37304048]